# Video evaluation

In [11]:
from pathlib import Path
import pyprojroot
dir_proj = pyprojroot.here()
print("Project directory:", dir_proj)

import json
import pandas as pd
import numpy as np

Project directory: /home/ck37/projects/ecig-vaping


## Review validation set videos

In [3]:
#dataset_version = "complete-in-review-2519" # Aug. 4, 2022
#dataset_version = "all-data-3258-items" # Oct. 24, 2022
dataset_version = "all-data-3287-items" # Oct. 24, 2022
dataset_name = "training-data"
dataset_id = "annotatevape/" + dataset_name + ":" + dataset_version
print("Dataset id:", dataset_id)

Dataset id: annotatevape/training-data:all-data-3287-items


In [4]:
# TODO: created a symlink within the project.
dir_base = Path("/home/ck37/.darwin/datasets/annotatevape/" + dataset_name + "/releases/")
dir_labels = dir_base / dataset_version

!ls {dir_labels}

annotations  lists


In [5]:
dir_videos = Path("/home/ck37/.darwin/datasets/annotatevape/" + dataset_name + "/images/")
print("Video directory:", dir_videos)

# Extract the filenames for the mp4s in our images directory.
files_videos = [video.stem for video in dir_videos.glob('*.mp4')]
print(f"mp4s found ({len(files_videos)}):")
print(", ".join([video for video in files_videos]))

Video directory: /home/ck37/.darwin/datasets/annotatevape/training-data/images
mp4s found (50):
#12, #5, #28, #16, several_pods_2, box_mod_ejuice, eliquid, multiple_pods, #26, #14, #23, #4, pods_clouds2, #6, #13, mod_ejuice, pod_clouds, #17, #21, #8, #19, #25, #27, #22, #18, box_pod_mod, #3, #7, #10, no_warning_label2_MANIPULATE, #29, several_pods, pod_cloud, #30, #20, two_pods, pods_clouds, boxes_pod_clouds, #2, box_pod, #24, #9, #15, pods, ejuices, multiplepods_clouds, ejuice_cloud_mod, #1, boxes, #11


In [6]:
files_videos

['#12',
 '#5',
 '#28',
 '#16',
 'several_pods_2',
 'box_mod_ejuice',
 'eliquid',
 'multiple_pods',
 '#26',
 '#14',
 '#23',
 '#4',
 'pods_clouds2',
 '#6',
 '#13',
 'mod_ejuice',
 'pod_clouds',
 '#17',
 '#21',
 '#8',
 '#19',
 '#25',
 '#27',
 '#22',
 '#18',
 'box_pod_mod',
 '#3',
 '#7',
 '#10',
 'no_warning_label2_MANIPULATE',
 '#29',
 'several_pods',
 'pod_cloud',
 '#30',
 '#20',
 'two_pods',
 'pods_clouds',
 'boxes_pod_clouds',
 '#2',
 'box_pod',
 '#24',
 '#9',
 '#15',
 'pods',
 'ejuices',
 'multiplepods_clouds',
 'ejuice_cloud_mod',
 '#1',
 'boxes',
 '#11']

In [7]:
# Identify which videos we have annotations for.
dir_annotations = dir_labels / "annotations"

# Extract all of the annotations we currently have for this dataset version.
all_annotations = [file.stem for file in dir_annotations.glob('*.json')]

# The intersection of the annotaitons and video files lists will give us the correct set of videos/annotations.
annotated_videos = list(set(files_videos) & set(all_annotations))
print(f'Found {len(annotated_videos)} annotated videos:\n', annotated_videos)

Found 50 annotated videos:
 ['#17', '#8', '#23', 'pods', 'pod_clouds', '#30', 'eliquid', 'ejuice_cloud_mod', '#1', 'pods_clouds2', '#6', 'boxes', 'ejuices', 'mod_ejuice', 'no_warning_label2_MANIPULATE', '#9', '#24', 'several_pods_2', '#16', '#5', '#20', 'boxes_pod_clouds', '#13', '#28', '#27', 'several_pods', 'box_pod', 'pod_cloud', '#29', 'two_pods', 'box_pod_mod', '#21', '#12', '#10', 'multiplepods_clouds', '#22', 'multiple_pods', '#4', '#3', '#15', '#19', '#7', '#11', '#18', 'pods_clouds', '#26', '#14', '#25', 'box_mod_ejuice', '#2']


In [12]:

video_dfs = {}

# Object classes to analyze
#target_class = ('mod', 'pod', 'e-juice', 'box', 'smoke cloud')
target_class = ('mod', 'pod', 'e-juice', 'box', 'smoke cloud', 'e-cigarette brand name', 'warning label nicotine', 'e-juice flavor')
#('synthetic nicotine label', )

# Loop over each video and collect metadata
for video in annotated_videos:
    print("Analyzing", video)
    # Tally the annotation statistics for each video
    annotation_file = dir_annotations / str(video + ".json")
    
    if not annotation_file.is_file():
        print("Could not find annotation for", video)
        continue
    
    # Import the annotation json.
    ann = json.load(open(annotation_file, 'r'))
    
    filename = ann['image']['filename']
    
    frame_count = ann['image']['frame_count']
    print("Filename:", filename, "Frame count:", frame_count)
    
    # Define the columns that we want to track
    df_columns = (
        'video',
        'frame'
    ) + target_class
    
    # Create a dataframe to store the frame counts for each object.
    video_df = \
        pd.DataFrame(0,
                     index = np.arange(frame_count),
                     columns = df_columns)
    
    video_df['frame'] = np.arange(frame_count)
    video_df['video'] = video

#    print(video_df)
            
    annotation_len = len(ann["annotations"])
    print("Annotation len:", annotation_len)
    
    if 'exclude' in ann['annotations']:
        print("Skipping this excluded video")
        continue
        
    if filename == "multiple_pods.mp4":
        #print(ann['annotations'])
        print(annotation_file)
    
    # Each annotation element has a certain frame range
    for ann_i in ann['annotations']:
        
        # Skip if this is not a relevant annotation
        if not ann_i['name'] in target_class:
            continue
        
        # Unclear why the above isn't triggered this correctly.
        if ann_i['name'] == 'exclude':
            print("Skipping an exclusion")
            continue
        
        segments = ann_i['segments']
        #print(ann_i['name'], "Segments:", segments)
        
        # Loop over each annotated frame
        for frame_i, val in ann_i['frames'].items():
            if ann_i['name'] == 'exclude':
                print("Frame i:", frame_i)
                print("Ann i:\n", ann_i)
            new_val = video_df.at[int(frame_i), ann_i['name']] + 1
            #print("Frame i", frame_i, "Val:", new_val)
            # Increment the count of the object type for that frame.
            video_df.at[int(frame_i), ann_i['name']] = new_val
    
    print()
    video_dfs[video] = video_df
    #for 

# Apply the model to each video frame
# Analyze the accurate of the model predictions

Analyzing two_pods
Filename: two_pods.mp4 Frame count: 99
Annotation len: 15

Analyzing boxes
Filename: boxes.mp4 Frame count: 68
Annotation len: 7

Analyzing #9
Filename: #9.mp4 Frame count: 157
Annotation len: 16

Analyzing #16
Filename: #16.mp4 Frame count: 198
Annotation len: 25

Analyzing #15
Filename: #15.mp4 Frame count: 108
Annotation len: 3

Analyzing #11
Filename: #11.mp4 Frame count: 93
Annotation len: 26

Analyzing #2
Filename: #2.mp4 Frame count: 130
Annotation len: 36

Analyzing #23
Filename: #23.mp4 Frame count: 435
Annotation len: 29

Analyzing #13
Filename: #13.mp4 Frame count: 204
Annotation len: 29

Analyzing mod_ejuice
Filename: mod_ejuice.mp4 Frame count: 217
Annotation len: 45

Analyzing box_pod_mod
Filename: box_pod_mod.mp4 Frame count: 107
Annotation len: 18

Analyzing #25
Filename: #25.mp4 Frame count: 111
Annotation len: 21

Analyzing #22
Filename: #22.mp4 Frame count: 154
Annotation len: 8

Analyzing #28
Filename: #28.mp4 Frame count: 64
Annotation len: 11

A

In [13]:
video_dfs.keys()

dict_keys(['two_pods', 'boxes', '#9', '#16', '#15', '#11', '#2', '#23', '#13', 'mod_ejuice', 'box_pod_mod', '#25', '#22', '#28', '#19', '#17', '#10', 'multiplepods_clouds', '#29', 'pods', 'eliquid', 'pod_clouds', '#4', 'no_warning_label2_MANIPULATE', 'pods_clouds', '#5', 'pod_cloud', '#6', 'box_pod', '#1', '#18', 'ejuices', 'ejuice_cloud_mod', '#14', '#30', '#21', 'pods_clouds2', 'multiple_pods', '#3', '#26', '#20', 'boxes_pod_clouds', 'several_pods_2', '#7', '#8', '#27', 'box_mod_ejuice', '#24', 'several_pods', '#12'])

In [14]:
video_dfs['several_pods']

,video,frame,mod,pod,e-juice,box,smoke cloud,e-cigarette brand name,warning label nicotine,e-juice flavor
0,several_pods,0,0,4,0,0,0,0,0,0
1,several_pods,1,0,4,0,0,0,0,0,0
2,several_pods,2,0,5,0,0,0,0,0,0
3,several_pods,3,0,5,0,0,0,0,0,0
4,several_pods,4,0,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
69,several_pods,69,0,2,0,0,1,0,0,0
70,several_pods,70,0,2,0,0,1,0,0,0
71,several_pods,71,0,2,0,0,1,0,0,0
72,several_pods,72,0,3,0,0,1,0,0,0


In [15]:
df = pd.concat(video_dfs)

In [16]:
df.shape

(5768, 10)

In [26]:
# This object isn't being used anymore.
"""
types = ('mod',
        'pod',
        'e-juice',
        'box',
        'smoke cloud')
"""

In [17]:
df['num_objects'] = df[[*target_class]].sum(axis = 1)
df['any_object'] = df['num_objects'] > 0

In [18]:
df.head()

video  frame  mod  pod  e-juice  box  smoke cloud  \
two_pods 0  two_pods      0    0    2        0    0            0   
         1  two_pods      1    0    2        0    0            0   
         2  two_pods      2    0    2        0    0            0   
         3  two_pods      3    0    2        0    0            0   
         4  two_pods      4    0    2        0    0            0   

            e-cigarette brand name  warning label nicotine  e-juice flavor  \
two_pods 0                       0                       0               0   
         1                       0                       0               0   
         2                       0                       0               0   
         3                       0                       0               0   
         4                       0                       0               0   

            num_objects  any_object  
two_pods 0            2        True  
         1            2        True  
         2            2        True  
         3            2        True  
         4            2        True

In [19]:
# Only 192 (3.3%) frames had no objects
df['any_object'].value_counts()

True     5576
False     192
Name: any_object, dtype: int64

In [30]:
5576 / 5768

0.9667128987517337

### Save summary statistics

In [20]:
df.to_parquet(str(dir_proj / "data/video-eval-summary-stats.parquet"), engine = "pyarrow")
# TODO: make a separate tables/ subdir somewhere.
df.to_excel(str(dir_proj / "data/video-eval-summary-stats.xlsx"))

## Apply model to videos

Here we have the option of scoring the whole video and then extracting specific frames, or just scoring the annotated frames.

It may be slightly preferable to score the full video and then analyze specific frames, because with that version we could potentially do smoothing or other post-processing leveraging the prediction information on frames that weren't annotated.file_checkpoint = 'work_dirs/' + model_name + "-" + timestr +'/latest.pth'


In [8]:
# Copied from mmdet-train.ipynb
model_name = "vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco"
dir_models = Path("../models/")

path_model = dir_models / str(model_name + ".py")
print("Model file:", path_model)
print("Find model:", path_model.is_file())
timestr = "20220523-105448"

file_checkpoint = dir_proj / ('notebooks/work_dirs/' + model_name + "-" + timestr +'/latest.pth')
print("Checkpoint file:", file_checkpoint)
print("Find checkpoint:", file_checkpoint.is_file())

Model file: ../models/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py
Find model: True
Checkpoint file: /home/ck37/projects/ecig-vaping/notebooks/work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco-20220523-105448/latest.pth
Find checkpoint: True


In [33]:
# Modified from mmdet-train.ipynb
import os

def predict_video(video_name, video_dir,
                  dir_output = pyprojroot.here() / "data/detections/videos",
                  box_threshold = 0.4, verbose = False):
    print("Analyzing", video_name)

    video_file = video_dir / video_name
    
    if not os.path.exists(video_file):
        print("Could not find video file")
        raise Exception
            
    path_out = dir_output / Path(video_file).name
    if verbose:
        print("Output path:", path_out)
    
    # Default threshold is 0.3
#    !python {pyprojroot.here()}/external/mmdetection/demo/video_demo.py "{video_file}" \
    !python {pyprojroot.here()}/notebooks/video_demo-ck.py "{video_file}" \
        {path_model} \
        {file_checkpoint} \
        --out "{path_out}" \
        --save_result \
        --score-thr {box_threshold}

In [ ]:
for video_name in files_videos:
    # Second argument is the probability threshold for showing a bounding box.
    predict_video(video_name + ".mp4", dir_videos, box_threshold = 0.4, verbose = True)

Analyzing #12.mp4
Output path: /home/ck37/projects/ecig-vaping/data/detections/videos/#12.mp4
load checkpoint from local path: /home/ck37/projects/ecig-vaping/notebooks/work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco-20220523-105448/latest.pth
2022-10-24 16:15:22,216 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2022-10-24 16:15:22,217 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2022-10-24 16:15:22,218 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2022-10-24 16:15:22,219 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2022-10-24 16:15:22,220 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2022-10-24 16:15:22,221 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2022-10-24 16:15:22,222 - root - INFO - ModulatedDeformConvP

In [ ]:
# Upload predicted to Google drive:
for video_name in files_videos:
    video_file = pyprojroot.here() / "data/detections/videos" / (video_name + ".mp4")
    if video_file.is_file():
        !rclone copy --progress {video_file} gdrive:deep_learning/detections/video-evaluation-paper
    else:
        print("Could not find prediction video for:", video_name)

## Apply model to individual annotated frames

This may be more accurate than trying to identify which annotated frame corresponds to which specific frame in a full video.

Each video will have a directory based on its filename that contains each annotated frame.

### Setup model for inference

In [9]:
from mmdet.apis import init_detector, inference_detector
import mmcv

# Specify the path to model config and checkpoint file
# This are specified above.

# Specify device to use - 2nd GPU here.
model = init_detector(str(path_model), str(file_checkpoint), device='cuda:0')

load checkpoint from local path: /home/ck37/projects/ecig-vaping/notebooks/work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco-20220523-105448/latest.pth


2023-05-02 07:44:31,845 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2023-05-02 07:44:31,847 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2023-05-02 07:44:31,848 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2023-05-02 07:44:31,849 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2023-05-02 07:44:31,851 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2023-05-02 07:44:31,853 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2023-05-02 07:44:31,855 - root - INFO - ModulatedDeformConvPack backbone.layer3.2.conv2 is upgraded to version 2.
2023-05-02 07:44:31,856 - root - INFO - ModulatedDeformConvPack backbone.layer3.3.conv2 is upgraded to version 2.
2023-05-02 07:44:31,858 - root - INFO - ModulatedDeformConvPack backbone.layer3.4.conv2 

### Inference on frames

In [25]:
from tqdm import tqdm
import pickle

# Run on all images
extensions = {".png", ".jpg"}

path_out = pyprojroot.here() / "data/detections/videos"

video_dfs = {}
detection_threshold = 0.4

#for video_name in files_videos[:2]:
for video_name in files_videos:
    print("Analyzing", video_name)
    # See if we can find the directory with annotated frames.
    video_frame_dir = dir_videos / video_name
    if video_frame_dir.is_dir():
        
        # Identify the frame images in the video's directory.
        files = [p for p in video_frame_dir.glob("*") if p.suffix in extensions]

        print(f"Found {len(files)} files to analyze.")

        det_results = {}
        
        # Define the columns that we want to track
        """df_columns = {
                'video',
                'frame',
                'mod',
                'pod',
                'e-juice',
                'box',
                'smoke cloud'
        }"""
        df_columns = (
                'video',
                'frame',                
            ) + target_class
        
        
        # Create a dataframe to store the frame counts for each object.
        video_df = \
            pd.DataFrame(0,
                         index = np.arange(len(files)),
                         columns = df_columns)

        video_df['frame'] = np.arange(len(files))
        video_df['video'] = video_name 
        
        # Loop over the frame files
        for frame_i in tqdm(files):
            result = inference_detector(model, frame_i)
            #det_results[file_i.stem] = result
            
            # Each element is one of the 9 classes.
            # Each subelement is a bounding box for that class, along with classification probability.
            # The specific class ordering is specified in the model config file (e.g. models/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py)
            # Current class order:
            classes = ('box', 'e-cigarette brand name', 'e-juice', 'e-juice flavor', 'mod', 'pod', 'smoke cloud', 'synthetic nicotine label', 'warning label nicotine')
            
            # These are the object detections that we want to count
            """pred_types = ('mod',
                          'pod',
                          'e-juice',
                          'box',
                          'smoke cloud')
            """
            pred_types = target_class
            
            # Defined much higher in the script.
            pred_types = target_class
            
            # Loop over each prediction type that we care about and increment the count if the probability exceeds the threshold
            for pred_type_i in pred_types:
                # Find the location of this object type in the model result object.
                pred_index = classes.index(pred_type_i)
                
                # Count the number of predictions for this object that exceed the probability threshold for detection.
                found_count = len([1 for pred_i in result[pred_index] if pred_i[4] > detection_threshold])
                video_df.at[int(frame_i.stem), pred_type_i] = found_count

        with open((path_out / video_name).with_suffix('.pkl'), 'wb') as file:
            pickle.dump(det_results, file, protocol = pickle.HIGHEST_PROTOCOL)
        
        # Save the count dataframe.
        video_df.to_parquet((path_out / video_name).with_suffix('.parquet'))
        video_dfs[video_name] = video_df
    
    else:
        print("Could not find annotated frame directory for:", video_name)
        
# Combine all predictions into a single dataframe.
pred_df = pd.concat(video_dfs)
pred_df.to_parquet(path_out / "video-eval-preds.parquet")

Analyzing #12
Could not find annotated frame directory for: #12
Analyzing #5
Could not find annotated frame directory for: #5
Analyzing #28
Could not find annotated frame directory for: #28
Analyzing #16
Could not find annotated frame directory for: #16
Analyzing several_pods_2
Found 24 files to analyze.


100%|███████████████████████████████████████████████████| 24/24 [00:05<00:00,  4.40it/s]


Analyzing box_mod_ejuice
Found 72 files to analyze.


100%|███████████████████████████████████████████████████| 72/72 [00:12<00:00,  5.57it/s]


Analyzing eliquid
Found 42 files to analyze.


...

Analyzing ejuice_cloud_mod
Found 61 files to analyze.


100%|███████████████████████████████████████████████████| 61/61 [00:14<00:00,  4.21it/s]


Analyzing #1
Could not find annotated frame directory for: #1
Analyzing boxes
Found 68 files to analyze.


100%|███████████████████████████████████████████████████| 68/68 [00:16<00:00,  4.15it/s]

Analyzing #11
Could not find annotated frame directory for: #11


In [26]:
pred_df.head()

video  frame  mod  pod  e-juice  box  smoke cloud  \
several_pods_2 0  several_pods_2      0    0    0        1    1            0   
               1  several_pods_2      1    0    1        1    0            0   
               2  several_pods_2      2    0    1        1    0            0   
               3  several_pods_2      3    0    1        1    0            0   
               4  several_pods_2      4    0    1        1    0            0   

                  e-cigarette brand name  warning label nicotine  \
several_pods_2 0                       1                       0   
               1                       1                       0   
               2                       2                       0   
               3                       4                       0   
               4                       3                       0   

                  e-juice flavor  
several_pods_2 0               1  
               1               1  
               2               1  
               3               0  
               4               0

## Compare predictions to annotations

In [27]:
# Join annotated df to predicted df
df2 = df.join(pred_df, on = ("video", "frame"), rsuffix = "_pred")

In [28]:
df2.head()
df2.to_excel(path_out / "video-eval-comparison.xlsx")

In [30]:
#pred_types = ('mod', 'pod', 'e-juice', 'box', 'smoke cloud')
pred_types = target_class
# For each object, calculate the absolute error for each frame

result_df = pd.DataFrame(0, index = [0], columns = pred_types)
            
for pred_type_i in pred_types:
    # |actual - predicted|
    mae = np.mean(np.abs(df2[pred_type_i].values - df2[pred_type_i + "_pred"]))
    result_df.at[0, pred_type_i] = mae

In [31]:
result_df

,mod,pod,e-juice,box,smoke cloud,e-cigarette brand name,warning label nicotine,e-juice flavor
0,0.246521,0.316103,0.115971,0.114645,0.535454,0.455268,0.180252,0.292247


In [32]:
# Then calculate the final summary stats.
df['num_objects_pred'] = df[[*pred_types]].sum(axis = 1)
df['any_object_pred'] = df['num_objects_pred'] > 0

In [33]:
# Of frames with any object, what percentage did the model flag as having any object?
print(np.mean(df.loc[df.any_object].any_object_pred.values))
# Of frames without an object, what percentage did the model correctly flag as not having an object?
print(np.mean(~ df.loc[~df.any_object.values].any_object_pred.values))

1.0
1.0


In [34]:
# 192 frames with no objects.
print(df.loc[~ df.any_object.values].shape)
df.loc[~ df.any_object.values]

(192, 14)


video  frame  mod  pod  e-juice  box  \
boxes          25            boxes     25    0    0        0    0   
               26            boxes     26    0    0        0    0   
               27            boxes     27    0    0        0    0   
               28            boxes     28    0    0        0    0   
               29            boxes     29    0    0        0    0   
...                            ...    ...  ...  ...      ...  ...   
#27            30              #27     30    0    0        0    0   
box_mod_ejuice 44   box_mod_ejuice     44    0    0        0    0   
#24            52              #24     52    0    0        0    0   
#12            103             #12    103    0    0        0    0   
               134             #12    134    0    0        0    0   

                    smoke cloud  e-cigarette brand name  \
boxes          25             0                       0   
               26             0                       0   
               27             0                       0   
               28             0                       0   
               29             0                       0   
...                         ...                     ...   
#27            30             0                       0   
box_mod_ejuice 44             0                       0   
#24            52             0                       0   
#12            103            0                       0   
               134            0                       0   

                    warning label nicotine  e-juice flavor  num_objects  \
boxes          25                        0               0            0   
               26                        0               0            0   
               27                        0               0            0   
               28                        0               0            0   
               29                        0               0            0   
...                                    ...             ...          ...   
#27            30                        0               0            0   
box_mod_ejuice 44                        0               0            0   
#24            52                        0               0            0   
#12            103                       0               0            0   
               134                       0               0            0   

                    any_object  num_objects_pred  any_object_pred  
boxes          25        False                 0            False  
               26        False                 0            False  
               27        False                 0            False  
               28        False                 0            False  
               29        False                 0            False  
...                        ...               ...              ...  
#27            30        False                 0            False  
box_mod_ejuice 44        False                 0            False  
#24            52        False                 0            False  
#12            103       False                 0            False  
               134       False                 0            False  

[192 rows x 14 columns]

In [62]:
# 1462 frames with one or more objects.
print(df.loc[df.any_object.values].shape)
print(df.loc[df.any_object.values].any_object_pred.value_counts())
df.loc[df.any_object.values]

(1462, 11)
True    1462
Name: any_object_pred, dtype: int64


mod             video  e-juice  frame  smoke cloud  box  \
several_pods_2   0     0    several_pods_2        0      0            1    0   
                 1     0    several_pods_2        0      1            1    0   
                 2     0    several_pods_2        0      2            1    0   
                 3     0    several_pods_2        0      3            1    0   
                 4     0    several_pods_2        0      4            1    0   
...                  ...               ...      ...    ...          ...  ...   
ejuice_cloud_mod 55    1  ejuice_cloud_mod        1     55            1    0   
                 56    1  ejuice_cloud_mod        1     56            1    0   
                 57    1  ejuice_cloud_mod        1     57            1    0   
                 58    1  ejuice_cloud_mod        1     58            1    0   
                 59    1  ejuice_cloud_mod        0     59            0    0   

                     pod  num_objects  any_object  num_objects_pred  \
several_pods_2   0     2            3        True                 3   
                 1     2            3        True                 3   
                 2     2            3        True                 3   
                 3     2            3        True                 3   
                 4     2            3        True                 3   
...                  ...          ...         ...               ...   
ejuice_cloud_mod 55    0            3        True                 3   
                 56    0            3        True                 3   
                 57    0            3        True                 3   
                 58    0            3        True                 3   
                 59    0            1        True                 1   

                     any_object_pred  
several_pods_2   0              True  
                 1              True  
                 2              True  
                 3              True  
                 4              True  
...                              ...  
ejuice_cloud_mod 55             True  
                 56             True  
                 57             True  
                 58             True  
                 59             True  

[1462 rows x 11 columns]

## Extract object probabilities

May 2023

In [20]:
from tqdm import tqdm
import pickle

# Run on all images
extensions = {".png", ".jpg"}

path_out = pyprojroot.here() / "data/detections/videos"

video_dfs = {}

for video_name in files_videos:
    print("Analyzing", video_name)
    # See if we can find the directory with annotated frames.
    video_frame_dir = dir_videos / video_name
    if video_frame_dir.is_dir():
        
        # Identify the frame images in the video's directory.
        files = [p for p in video_frame_dir.glob("*") if p.suffix in extensions]

        print(f"Found {len(files)} files to analyze.")
     
        all_frames = []
        
        # Loop over the frame files
        for frame_i in tqdm(files):
            result = inference_detector(model, frame_i)
            
            # Create a dataframe for each video frame
                            
            #det_results[file_i.stem] = result
            
            # Each element is one of the 9 classes.
            # Each subelement is a bounding box for that class, along with classification probability.
            # The specific class ordering is specified in the model config file (e.g. models/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py)
            # Current class order:
            classes = ('box', 'e-cigarette brand name', 'e-juice', 'e-juice flavor', 'mod', 'pod', 'smoke cloud', 'synthetic nicotine label', 'warning label nicotine')
                        
            # Defined much higher in the script.
            pred_types = classes

            frame_results = []
            
            # Loop over each prediction type that we care about and increment the count if the probability exceeds the threshold
            for pred_type_i in pred_types:
                # Find the location of this object type in the model result object.
                pred_index = classes.index(pred_type_i)
                
                for pred_i in result[pred_index]:
                    frame_results.append({'frame': frame_i.stem,
                                          'class': pred_type_i,
                                          'prob': pred_i[4]})
            
            frame_df = pd.DataFrame(frame_results)
            all_frames.append(frame_df)
            
        video_df = pd.concat(all_frames)
        video_df['video'] = video_name
        
        print(video_df)

        # Save the prob dataframe.
        out_name = (path_out / str(video_name + "-probs")).with_suffix('.parquet')
        print(out_name)
        video_df.to_parquet(out_name)
        video_dfs[video_name] = video_df
    
    else:
        print("Could not find annotated frame directory for:", video_name)
        
# Combine all predictions into a single dataframe.
pred_df = pd.concat(video_dfs)
pred_df.to_parquet(path_out / "video-eval-preds-probs.parquet")

Analyzing #12
Could not find annotated frame directory for: #12
Analyzing #5
Could not find annotated frame directory for: #5
Analyzing #28
Could not find annotated frame directory for: #28
Analyzing #16
Could not find annotated frame directory for: #16
Analyzing several_pods_2
Found 24 files to analyze.


100%|█████████████████████████████████████████████| 24/24 [00:03<00:00,  6.77it/s]


      frame                   class      prob           video
0   0000010  e-cigarette brand name  0.750092  several_pods_2
1   0000010  e-cigarette brand name  0.691061  several_pods_2
2   0000010  e-cigarette brand name  0.636141  several_pods_2
3   0000010  e-cigarette brand name  0.211906  several_pods_2
4   0000010  e-cigarette brand name  0.098541  several_pods_2
..      ...                     ...       ...             ...
35  0000012             smoke cloud  0.057404  several_pods_2
36  0000012             smoke cloud  0.051671  several_pods_2
37  0000012  warning label nicotine  0.217161  several_pods_2
38  0000012  warning label nicotine  0.153057  several_pods_2
39  0000012  warning label nicotine  0.099991  several_pods_2

[718 rows x 4 columns]
/home/ck37/projects/ecig-vaping/data/detections/videos/several_pods_2-probs.parquet
Analyzing box_mod_ejuice
Found 72 files to analyze.


100%|█████████████████████████████████████████████| 72/72 [00:10<00:00,  6.57it/s]


      frame                     class      prob           video
0   0000046    e-cigarette brand name  0.109331  box_mod_ejuice
1   0000046    e-cigarette brand name  0.097620  box_mod_ejuice
2   0000046            e-juice flavor  0.115360  box_mod_ejuice
3   0000046                       mod  0.090411  box_mod_ejuice
4   0000046                       pod  0.304536  box_mod_ejuice
..      ...                       ...       ...             ...
34  0000012  synthetic nicotine label  0.092339  box_mod_ejuice
35  0000012    warning label nicotine  0.901507  box_mod_ejuice
36  0000012    warning label nicotine  0.144617  box_mod_ejuice
37  0000012    warning label nicotine  0.121660  box_mod_ejuice
38  0000012    warning label nicotine  0.069118  box_mod_ejuice

[1207 rows x 4 columns]
/home/ck37/projects/ecig-vaping/data/detections/videos/box_mod_ejuice-probs.parquet
Analyzing eliquid
Found 42 files to analyze.


...

      frame        class      prob                video
0   0000046      e-juice  0.054797  multiplepods_clouds
1   0000046          mod  0.143012  multiplepods_clouds
2   0000046          mod  0.141727  multiplepods_clouds
3   0000046          pod  0.576364  multiplepods_clouds
4   0000046          pod  0.366702  multiplepods_clouds
..      ...          ...       ...                  ...
3   0000012          pod  0.150330  multiplepods_clouds
4   0000012          pod  0.050726  multiplepods_clouds
5   0000012  smoke cloud  0.062139  multiplepods_clouds
6   0000012  smoke cloud  0.056302  multiplepods_clouds
7   0000012  smoke cloud  0.051946  multiplepods_clouds

[931 rows x 4 columns]
/home/ck37/projects/ecig-vaping/data/detections/videos/multiplepods_clouds-probs.parquet
Analyzing ejuice_cloud_mod
Found 61 files to analyze.


100%|█████████████████████████████████████████████| 61/61 [00:10<00:00,  6.07it/s]


      frame                   class      prob             video
0   0000046             smoke cloud  0.054684  ejuice_cloud_mod
0   0000010  e-cigarette brand name  0.054002  ejuice_cloud_mod
1   0000010          e-juice flavor  0.073053  ejuice_cloud_mod
2   0000010             smoke cloud  0.981999  ejuice_cloud_mod
3   0000010             smoke cloud  0.165922  ejuice_cloud_mod
..      ...                     ...       ...               ...
4   0000012                     pod  0.076711  ejuice_cloud_mod
5   0000012             smoke cloud  0.959120  ejuice_cloud_mod
6   0000012             smoke cloud  0.283607  ejuice_cloud_mod
7   0000012             smoke cloud  0.088931  ejuice_cloud_mod
8   0000012             smoke cloud  0.068241  ejuice_cloud_mod

[1169 rows x 4 columns]
/home/ck37/projects/ecig-vaping/data/detections/videos/ejuice_cloud_mod-probs.parquet
Analyzing #1
Could not find annotated frame directory for: #1
Analyzing boxes
Found 68 files to analyze.


100%|█████████████████████████████████████████████| 68/68 [00:11<00:00,  5.95it/s]

      frame                     class      prob  video
0   0000046                       box  0.305452  boxes
1   0000046                       box  0.057641  boxes
2   0000046    e-cigarette brand name  0.149133  boxes
3   0000046    e-cigarette brand name  0.102404  boxes
4   0000046    e-cigarette brand name  0.078932  boxes
..      ...                       ...       ...    ...
29  0000012               smoke cloud  0.055094  boxes
30  0000012  synthetic nicotine label  0.062537  boxes
31  0000012    warning label nicotine  0.947372  boxes
32  0000012    warning label nicotine  0.247777  boxes
33  0000012    warning label nicotine  0.127145  boxes

[1325 rows x 4 columns]
/home/ck37/projects/ecig-vaping/data/detections/videos/boxes-probs.parquet
Analyzing #11
Could not find annotated frame directory for: #11
